# Ch. 5 When to use object-oriented programming

"As the program expands, we will later find that we are passing the same set of related variables to a set of functions."

## Adding behavior to class data with properties

Propoerties make methods look like attributes

In [ ]:
class Color:
    def __init__(self, name):
        self._name = name
    
    def _set_name(self, name):
        if not name:
            raise ValueError("Invalid Name")
        self._name = name
    
    def _get_name(self):
        return self._name

# try get, set, get

In [ ]:
class Color:
    def __init__(self, name):
        self._name = name
    
    def _set_name(self, name):
        if not name:
            raise ValueError("Invalid Name")
        self._name = name
    
    def _get_name(self):
        return self._name
    
    name = property(_get_name, _set_name)

# try get, set, get (& raise our ValueError)

Why do this?

## Decorators (turn functions into properties)

"Clouding the division between behavior and data"

In [ ]:
class Color:
    def __init__(self, name):
        self._name = name

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        if not value:
            raise ValueError("Invalid Name")
        self._name = value

# get, set get

### When to use properties instead of attributes

"In general, always use a standard attribute
until you need to **control access** to that property in some way."

- also avoiding redundant computations

e.g., https://github.com/mapbox/rasterio/blob/87836436fe2b843ab723a391a519439008d882d5/rasterio/crs.py#L123

- also calculating things on the fly

e.g., https://github.com/pandas-dev/pandas/blob/145ade2a2f011dbfbe531347fd0dc563ee0b5642/pandas/core/arrays/datetimes.py#L1303

- also protecting users from themselves

e.g., https://github.com/pandas-dev/pandas/blob/145ade2a2f011dbfbe531347fd0dc563ee0b5642/pandas/core/arrays/datetimes.py#L551

## Re-using code

e.g., we need to 

1. Unzip a .zip file
2. Do some stuff
3. Zip the stuff

In [ ]:
import sys
import os
import shutil
import zipfile
from pathlib import Path

class ZipProcessor:
    def __init__(self, zipname):
        self.zipname = zipname
        self.tmpdir = Path('unzipped-{}'.format(zipname[:-4]))
    
    def process_zip(self):
        self.unzip_files()
        self.do_stuff()
        self.zip_files()
    
    def unzip_files(self):
        self.tmpdir.mkdir()
        with zipfile.ZipFile(self.zipname) as zip:
            zip.extractall(str(self.tmpdir))
    
    def zip_files(self):
        with zipfile.ZipFile(self.zipname, 'w') as file:
            for filename in self.tmpdir.iterdir():
                file.write(str(filename), filename.name)
        shutil.rmtree(str(self.tmpdir))


# create a subclass that does a find and replace
class ZipReplace(ZipProcessor):
    def __init__(self, filename, find, replace):
        super().__init__(filename)
        self.find = find
        self.replace = replace

    def do_stuff(self):
        '''perform a search and replace on all files in the
        temporary directory'''
        for filename in self.tmpdir.iterdir():
            with filename.open() as file:
                contents = file.read()
            contents = contents.replace(self.find, self.replace)
            with filename.open("w") as file:
                file.write(contents)

In [ ]:
zip_replacer = ZipReplace(filename = 'lyrics.zip', 
                          find = 'birthday', 
                          replace = 'friday')

zip_replacer.process_zip()